In [26]:
F193.<a> = GF(193) # 193 = 64 * 3 + 1
R193.<X> = F193[]
R193
Fp=F193

from utils import log_2, next_power_of_two, is_power_of_two, bits_le_with_width, inner_product, reverse_bits

def nth_root_of_unity(n):
    k = log_2(n)
    assert k <= 6, "k is greater than 6"
    
    return Fp(5)**(3*2**(6-k))

Fp.nth_root_of_unity = nth_root_of_unity
Fp.TWO_ADICITY = 6
Fp.MULTIPLICATIVE_GENERATOR = Fp.primitive_element()
Fp.ROOT_OF_UNITY = Fp(5)**3

def neg_one():
    return Fp(-1)
Fp.neg_one = neg_one

R_2.<X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, Y0, Y1, Y2, Y3, Y4, Y5, Y6, Y7, A0, A1, A2, B0, B1, B2> = Fp[]; R_2

Multivariate Polynomial Ring in X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, Y0, Y1, Y2, Y3, Y4, Y5, Y6, Y7, A0, A1, A2, B0, B1, B2 over Finite Field of size 193

In [2]:
from mle2 import MLEPolynomial
from unipoly2 import UniPolynomial, UniPolynomialWithFft
from transcript import MerlinTranscript
from merkle import MerkleTree

MLEPolynomial.set_field_type(Fp)
transcript = MerlinTranscript("test")
UniPolynomial.set_field_type(Fp)
UniPolynomialWithFft.set_field_type(Fp)


In [3]:
def rs_encode(f: list[Fp], coset: Fp, blowup_factor: int) -> list[Fp]:
    n = next_power_of_two(len(f))
    N = n * blowup_factor

    omega_Nth = Fp.nth_root_of_unity(N)
    k = log_2(N)
    vec = f + [Fp.zero()] * (N - len(f))
    return UniPolynomialWithFft.fft_coset_rbo(vec, coset, k, omega=omega_Nth)


In [4]:
def rs_generator_matrix(g, k, c):
    assert is_power_of_two(k), "k: %d is not a power of two" % k
    assert is_power_of_two(c), "c: %d is not a power of two" % c

    n = k * c
    w = Fp.nth_root_of_unity(n)
    G = []
    for i in range(n):
        G.append([w**(i*j) for j in range(k)])
    return G

In [5]:
G = rs_generator_matrix(Fp(1), 8, 2)
G[1]

[1, 64, 43, 50, 112, 27, 184, 3]

In [6]:
def matrix_mul_vec(G, m):
    assert len(G[0]) == len(m), "len(G[0]) != len(m)"
    return [inner_product(G[i], m, Fp(0)) for i in range(len(G))]
matrix_mul_vec(G, [1, 2, 3, 4, 5, 6, 7, 8])

[36, 176, 184, 143, 127, 167, 115, 93, 189, 113, 70, 59, 58, 51, 1, 171]

In [7]:
def matrix_transpose(M):
    if type(M[0]) == type([]):
        return [[M[j][i] for j in range(len(M))] for i in range(len(M[0]))]
    elif type(M) == type([Fp(0)]):
        return [[M[i]] for i in range(len(M))]
matrix_transpose([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), matrix_transpose([1, 2, 3])


([[1, 4, 7], [2, 5, 8], [3, 6, 9]], [[1], [2], [3]])

In [8]:
def matrix_mul_matrix(G, M):
    m = len(G)
    n = len(G[0])
    k = len(M[0])
    assert n == len(M), "len(G[0]) != len(M[0])"
    A = []
    for i in range(m):
        A.append([inner_product(G[i], [M[l][j] for l in range(n)], Fp(0)) for j in range(k)])
    return A
matrix_mul_matrix(G, matrix_transpose(G))

[[8, 98, 0, 126, 0, 89, 0, 192, 0, 3, 0, 106, 0, 69, 0, 97],
 [98, 0, 126, 0, 89, 0, 192, 0, 3, 0, 106, 0, 69, 0, 97, 8],
 [0, 126, 0, 89, 0, 192, 0, 3, 0, 106, 0, 69, 0, 97, 8, 98],
 [126, 0, 89, 0, 192, 0, 3, 0, 106, 0, 69, 0, 97, 8, 98, 0],
 [0, 89, 0, 192, 0, 3, 0, 106, 0, 69, 0, 97, 8, 98, 0, 126],
 [89, 0, 192, 0, 3, 0, 106, 0, 69, 0, 97, 8, 98, 0, 126, 0],
 [0, 192, 0, 3, 0, 106, 0, 69, 0, 97, 8, 98, 0, 126, 0, 89],
 [192, 0, 3, 0, 106, 0, 69, 0, 97, 8, 98, 0, 126, 0, 89, 0],
 [0, 3, 0, 106, 0, 69, 0, 97, 8, 98, 0, 126, 0, 89, 0, 192],
 [3, 0, 106, 0, 69, 0, 97, 8, 98, 0, 126, 0, 89, 0, 192, 0],
 [0, 106, 0, 69, 0, 97, 8, 98, 0, 126, 0, 89, 0, 192, 0, 3],
 [106, 0, 69, 0, 97, 8, 98, 0, 126, 0, 89, 0, 192, 0, 3, 0],
 [0, 69, 0, 97, 8, 98, 0, 126, 0, 89, 0, 192, 0, 3, 0, 106],
 [69, 0, 97, 8, 98, 0, 126, 0, 89, 0, 192, 0, 3, 0, 106, 0],
 [0, 97, 8, 98, 0, 126, 0, 89, 0, 192, 0, 3, 0, 106, 0, 69],
 [97, 8, 98, 0, 126, 0, 89, 0, 192, 0, 3, 0, 106, 0, 69, 0]]

In [9]:
# Test matrix_mul_matrix()
matrix_mul_matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]], [[1, 2, 3], [5, 6, 7], [9, 10, 11]])

[[38, 44, 50], [83, 98, 113], [128, 152, 176], [173, 13, 46]]

In [10]:
# Test matrix_mul_matrix()
matrix_mul_matrix([[X0, X1, X2], [X3, X4, X5]], [[Y0], [Y1], [Y2]])

[[X0*Y0 + X1*Y1 + X2*Y2], [X3*Y0 + X4*Y1 + X5*Y2]]

In [11]:
# Test rs_encode() and matrix_mul()

# rs_encode([1, 2, 3, 4, 5, 6, 7, 8], Fp(1), 8, 2) == matrix_mul(G, [1, 2, 3, 4, 5, 6, 7, 8])

## Ligerito protocol


1. blowup factor is `2`
2. number of queries is `2`
3. coset generator is `Fp(1)`

In [15]:
blowup_factor = 2
num_queries = 2
coset_gen = Fp(1)


In [16]:
# An MLE polynomial example, with length 32
evals_over_hypercube = [Fp(2), Fp(3), Fp(4), Fp(5), Fp(6), Fp(7), Fp(8), Fp(9), \
             Fp(10), Fp(11), Fp(12), Fp(13), Fp(14), Fp(15), Fp(16), Fp(17), \
             Fp(2), Fp(3), Fp(4), Fp(5), Fp(6), Fp(7), Fp(8), Fp(9), \
             Fp(10), Fp(11), Fp(12), Fp(13), Fp(14), Fp(15), Fp(16), Fp(17), \
            ]

point = [Fp(-1), Fp(2), Fp(1), Fp(2), Fp(2)]

f_mle = MLEPolynomial(evals_over_hypercube, 5)

evaluation = f_mle.evaluate(point)
evaluation


25

In [17]:
transcript = MerlinTranscript("test")

In [22]:
# Commit to the MLE polynomial

class Commitment:

    def __init__(self, tree: MerkleTree):
        self.tree = tree
        self.cm = tree.root
        self.root = tree.root

    def __repr__(self):
        return f"Commitment(len={len(self.tree.data)}, root={self.cm})"
    
def commit(f_mle: MLEPolynomial) -> Commitment:
    evals = f_mle.evals
    k = f_mle.num_var
    k2 = k // 2
    k1 = k - k2
    n1 = 2**k1
    n2 = 2**k2
    c_mat = []
    for i in range(n1):
        c_row = [evals[i*n2+j] for j in range(n2)]
        c_row_code = rs_encode(c_row, coset_gen, blowup_factor)
        c_mat.append(c_row_code)

    c_mat_T = matrix_transpose(c_mat)
    print(f"c_mat_T={c_mat_T}")
    cm_vec = [MerkleTree(c_mat_T[i]) for i in range(len(c_mat_T))]
    # print(f"cm_vec={[cm_vec[i].root for i in range(len(cm_vec))]})")
    return Commitment(MerkleTree([cm.root for cm in cm_vec]))

f_cm = commit(f_mle)
f_cm


c_mat_T=[[14, 30, 46, 62, 14, 30, 46, 62], [191, 191, 191, 191, 191, 191, 191, 191], [160, 160, 160, 160, 160, 160, 160, 160], [29, 29, 29, 29, 29, 29, 29, 29], [148, 157, 166, 175, 148, 157, 166, 175], [173, 103, 33, 156, 173, 103, 33, 156], [128, 13, 91, 169, 128, 13, 91, 169], [138, 137, 136, 135, 138, 137, 136, 135]]


Commitment(len=8, root=24aa527a7c934be3c684c9eae6fa42a46037df5fa41be695016622f16ed441ed)

### Prove the evaluation argument


In [23]:
[Fp(-3)*Fp(65), Fp(64)*Fp(65)]


[191, 107]

## Misc

In [141]:
blowup_factor = 2

def prove_eval(mle: MLEPolynomial, point: list[Fp], transcript: MerlinTranscript):
    assert mle.num_var == len(point), "mle.num_var != len(point)"

    evals = mle.evals
    k = mle.num_var
    k1_prime = 2
    k1 = k - k1_prime

    G = rs_generator_matrix(Fp(1), 2**k1, blowup_factor)
    X = []
    for i in range(2**k1):
        X.append([evals[i+j*2**k1] for j in range(2**k1_prime)])

    r_vec = [Fp.random_element() for _ in range(k1_prime)]

    eq_r = MLEPolynomial.eqs_over_hypercube(r_vec)

    X_code = matrix_mul_matrix(G, X)
    
    y = matrix_mul_vec(X, eq_r)

    return X, X_code, y, r_vec

def verify_eval(X_code, y, point: list[Fp], r_vec: list[Fp],transcript: MerlinTranscript):

    k = len(point)
    k1_prime = 2
    k1 = k - k1_prime

    G = rs_generator_matrix(Fp(1), 2**k1, blowup_factor)

    eq_r = MLEPolynomial.eqs_over_hypercube(r_vec)

    lhs = matrix_mul_vec(X_code, eq_r)
    rhs = matrix_mul_vec(G, y)
    return lhs, rhs


In [142]:
def matrix_geometric(M):
    return len(M), len(M[0])

In [143]:
X, X_code, y, r_vec = prove_eval(MLEPolynomial([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 4), [1, 2, 3, 4], transcript)

In [144]:
r_vec, matrix_geometric(X_code), matrix_geometric(X), y

([157, 150], (8, 4), (4, 4), [92, 93, 94, 95])

In [145]:
verify_eval(X_code, y, [1, 2, 3, 4], r_vec,transcript)

([181, 61, 160, 146, 191, 142, 29, 19], [181, 61, 160, 146, 191, 142, 29, 19])

In [146]:
X, X_code, y, r_vec  = prove_eval(MLEPolynomial([X1, X2, X3, X4, X5, X6, X7, X8], 3), [1, 2, 3], transcript)

In [147]:
X, X_code, y, r_vec

([[X1, X3, X5, X7], [X2, X4, X6, X8]],
 [[X1 + X2, X3 + X4, X5 + X6, X7 + X8],
  [X1 - 81*X2, X3 - 81*X4, X5 - 81*X6, X7 - 81*X8],
  [X1 - X2, X3 - X4, X5 - X6, X7 - X8],
  [X1 + 81*X2, X3 + 81*X4, X5 + 81*X6, X7 + 81*X8]],
 [40*X1 + 90*X3 - 10*X5 + 74*X7, 40*X2 + 90*X4 - 10*X6 + 74*X8],
 [164, 64])

In [148]:
verify_eval(X_code, y, [1, 2, 3], r_vec, transcript)

([40*X1 + 40*X2 + 90*X3 + 90*X4 - 10*X5 - 10*X6 + 74*X7 + 74*X8,
  40*X1 + 41*X2 + 90*X3 + 44*X4 - 10*X5 + 38*X6 + 74*X7 - 11*X8,
  40*X1 - 40*X2 + 90*X3 - 90*X4 - 10*X5 + 10*X6 + 74*X7 - 74*X8,
  40*X1 - 41*X2 + 90*X3 - 44*X4 - 10*X5 - 38*X6 + 74*X7 + 11*X8],
 [40*X1 + 40*X2 + 90*X3 + 90*X4 - 10*X5 - 10*X6 + 74*X7 + 74*X8,
  40*X1 + 41*X2 + 90*X3 + 44*X4 - 10*X5 + 38*X6 + 74*X7 - 11*X8,
  40*X1 - 40*X2 + 90*X3 - 90*X4 - 10*X5 + 10*X6 + 74*X7 - 74*X8,
  40*X1 - 41*X2 + 90*X3 - 44*X4 - 10*X5 - 38*X6 + 74*X7 + 11*X8])

## Section 5: Matrix-vector product with partial sumcheck

In [172]:
blowup_factor = 2
debug = 2
def prove_eval(mle: MLEPolynomial, point: list[Fp], transcript: MerlinTranscript):
    assert mle.num_var == len(point), "mle.num_var != len(point)"

    v = mle.evaluate(point)
    print(f"P> v = {v}")
    evals = mle.evals
    k = mle.num_var
    k1_prime = 2
    k1 = k - k1_prime

    G = rs_generator_matrix(Fp(1), 2**k1, blowup_factor)
    X = []
    for i in range(2**k1):
        X.append([evals[i+j*2**k1] for j in range(2**k1_prime)])
    if debug > 0:
        print(f"P> X = {X}")

    f = mle.evals
    eq = MLEPolynomial.eqs_over_hypercube(point)
    sumcheck_h_vec = []
    sum_checked = v
    r_vec = []
    for i in range(k1_prime):
        if debug > 0:
            print(f"P> sumcheck round {i}")
        half = len(f) // 2
        f_low = f[:half]
        f_high = f[half:]
        eq_low = eq[:half]
        eq_high = eq[half:]

        h_eval_at_0 = sum([f_low[j] * eq_low[j] for j in range(half)], Fp(0))
        h_eval_at_1 = sum([f_high[j] * eq_high[j] for j in range(half)], Fp(0))
        h_eval_at_2 = sum([ (2 * f_high[j] - f_low[j]) * (2 * eq_high[j] - eq_low[j]) for j in range(half)], Fp(0))

        h = [h_eval_at_0, h_eval_at_1, h_eval_at_2]
        sumcheck_h_vec.append(h)

        transcript.absorb(b"h(X)", h)

        if debug > 0:
            print(f"P> h = {h}")
        
        assert h_eval_at_0 + h_eval_at_1 == sum_checked, \
            f"h_eval_at_0 + h_eval_at_1 = {h_eval_at_0 + h_eval_at_1}, sum_checked = {sum_checked}"

        r = Fp.random_element()

        r_vec.append(r)

        # fold f

        f_folded = [(Fp(1) - r) * f_low[i] + r * f_high[i] for i in range(half)]
        eq_folded = [(Fp(1) - r) * eq_low[i] + r * eq_high[i] for i in range(half)]

        f = f_folded
        eq = eq_folded

        sum_checked = UniPolynomial.evaluate_from_evals(h, 
                r, [Fp(0), Fp(1), Fp(2)])

    X_code = matrix_mul_matrix(G, X)

    X_code_T = matrix_transpose(X_code)
    
    cm_vec = [MerkleTree(X_code_T[i]) for i in range(len(X_code_T))]

    print(cm_vec)

    eq_r = MLEPolynomial.eqs_over_hypercube(r_vec[::-1])

    y = matrix_mul_vec(X, eq_r)
    assert y == f_folded, f"y != f_folded, y = {y}, f_folded = {f_folded}"
    

    if debug > 1: 
        print(f"P> check folded code")
        X_code_folded = matrix_mul_vec(X_code, eq_r)
        print(X_code_folded)
        assert X_code_folded == matrix_mul_vec(G, f_folded), f"X_code_folded != X_code"
        print(f"P> check folded code passed")

    return X, X_code, y, r_vec

def verify_eval(X_code, y, point: list[Fp], r_vec: list[Fp], transcript: MerlinTranscript):

    k = len(point)
    k1_prime = 2
    k1 = k - k1_prime

    G = rs_generator_matrix(Fp(1), 2**k1, blowup_factor)

    eq_r = MLEPolynomial.eqs_over_hypercube(r_vec)

    lhs = matrix_mul_vec(X_code, eq_r)
    rhs = matrix_mul_vec(G, y)
    return lhs, rhs



In [173]:
X, X_code, y, r_vec  = prove_eval(MLEPolynomial([X1, X2, X3, X4, X5, X6, X7, X8], 3), [1, 2, 3], transcript)

P> v = 2*X2 - 4*X4 - 3*X6 + 6*X8
P> X = [[X1, X3, X5, X7], [X2, X4, X6, X8]]
P> sumcheck round 0
P> h = [2*X2 - 4*X4, -3*X6 + 6*X8, 8*X2 - 16*X4 - 16*X6 + 32*X8]
P> sumcheck round 1
P> h = [2*X2, -4*X4, 10*X2 - 20*X4]
[<merkle.MerkleTree object at 0x161c27440>, <merkle.MerkleTree object at 0x161c27f80>, <merkle.MerkleTree object at 0x161c27620>, <merkle.MerkleTree object at 0x161c26480>]
P> check folded code
[56*X1 + 56*X2 - 55*X3 - 55*X4, 56*X1 + 96*X2 - 55*X3 + 16*X4, 56*X1 - 56*X2 - 55*X3 + 55*X4, 56*X1 - 96*X2 - 55*X3 - 16*X4]
P> check folded code passed


In [174]:
def reverse_bits(n: int, bit_length: int) -> int:
    """
    Reverse the bits of an integer.

    Args:
        n (int): The input integer.
        bit_length (int): The number of bits to consider.

    Returns:
        int: The integer with its bits reversed.
    """
    result = 0
    for i in range(bit_length):
        result = (result << 1) | (n & 1)
        n >>= 1
    return result

In [177]:
reverse_bits(4, 3)

1

In [124]:
def tensor_vector(index: int, k1: int, blowup_factor: int):
    N = 2**k1 * blowup_factor
    omega_Nth = Fp.nth_root_of_unity(N)
    index_rev = reverse_bits(index, log_2(N))
    omega = omega_Nth**index_rev
    print(f"omega = {omega}")
    vec = [omega**(2**i) for i in range(k1)]
    print(f"vec = {vec}")
    return vec

def rs_generator_matrix_at_row(index:int, f_len: int, coset:Fp, blowup_factor:int) -> list[Fp]:
    n = next_power_of_two(f_len)
    N = n * blowup_factor
    omega_Nth = Fp.nth_root_of_unity(N)
    index_rev = reverse_bits(index, log_2(N))
    omega = omega_Nth**index_rev
    return [coset* omega**i for i in range(n)]

In [125]:
tensor_vector(1, 3, 2)

omega = 192
vec = [192, 1, 1]


[192, 1, 1]

In [126]:
omega_Nth = Fp.nth_root_of_unity(16)
index_rev = reverse_bits(1, 4)
omega = omega_Nth**index_rev
index_rev, omega

(8, 192)

In [127]:
omega, omega**2, omega**4

(192, 1, 1)

In [128]:
[63 * Fp(93), -2 * Fp(93)]

[69, 7]

In [129]:
eq0 = [Fp(-83), Fp(28)]
rs_w = [Fp(1), Fp(-81)]
beta = Fp(93)
eq = [eq0[j] + beta * rs_w[j] for j in range(len(eq0))]
eq


[10, 22]

In [155]:
point = [Fp(3), Fp(2), Fp(2)]
k_last = 1
r_vec_all = [Fp(76), Fp(70)]

scalar = MLEPolynomial.evaluate_eq_polynomial(point[k_last:], r_vec_all)
eq_0 = MLEPolynomial.eqs_over_hypercube(point[:k_last])
eq_0 = [eq_0[j] * scalar for j in range(2**k_last)]
eq_0

[70, 88]

In [131]:
q=7
k1_prime = 1
k1 = 2

In [132]:
rs_generator_matrix_at_row(q, 2**k1, Fp(1), 4)

[1, 9, 81, 150]

In [133]:
r_vec = r_vec_all[:k1-k_last]
r_vec


[76]

In [147]:
def tensor_vector_with_monomial_basis(vec: list[Fp]) -> list[Fp]:
    if len(vec) == 0:
        return [Fp(1)]
    v = vec[-1]
    vec_expanded = tensor_vector_with_monomial_basis(vec[:-1])
    vec_right = [vec_expanded[i] * v for i in range(len(vec_expanded))]
    return vec_expanded + vec_right
tensor_vector_with_monomial_basis([X0, X1, X2])

[1, X0, X1, X0*X1, X2, X0*X2, X1*X2, X0*X1*X2]

In [148]:
def tensor_vector_with_multilinear_basis(vec: list[Fp]) -> list[Fp]:
    if len(vec) == 0:
        return [Fp(1)]
    v = vec[-1]
    vec_expanded = tensor_vector_with_multilinear_basis(vec[:-1])
    vec_left = [vec_expanded[i] * (Fp(1)-v) for i in range(len(vec_expanded))]
    vec_right = [vec_expanded[i] * v for i in range(len(vec_expanded))]
    return vec_left + vec_right
tensor_vector_with_multilinear_basis([X0, X1, X2])

[-X0*X1*X2 + X0*X1 + X0*X2 + X1*X2 - X0 - X1 - X2 + 1,
 X0*X1*X2 - X0*X1 - X0*X2 + X0,
 X0*X1*X2 - X0*X1 - X1*X2 + X1,
 -X0*X1*X2 + X0*X1,
 X0*X1*X2 - X0*X2 - X1*X2 + X2,
 -X0*X1*X2 + X0*X2,
 -X0*X1*X2 + X1*X2,
 X0*X1*X2]

In [149]:
MLEPolynomial.eqs_over_hypercube([X0, X1, X2])

[-X0*X1*X2 + X0*X1 + X0*X2 + X1*X2 - X0 - X1 - X2 + 1,
 X0*X1*X2 - X0*X1 - X0*X2 + X0,
 X0*X1*X2 - X0*X1 - X1*X2 + X1,
 -X0*X1*X2 + X0*X1,
 X0*X1*X2 - X0*X2 - X1*X2 + X2,
 -X0*X1*X2 + X0*X2,
 -X0*X1*X2 + X1*X2,
 X0*X1*X2]

In [161]:
# iteration 0

from functools import reduce
from operator import mul

beta=Fp(86)
q=7
k1_prime = 1
k1 = 2
r_vec = r_vec_all[:k1-k_last]

w_rev = tensor_vector(q, k1, 2)
print(f"w_rev = {w_rev}")
w_ex = tensor_vector_with_monomial_basis(w_rev[k_last:])
print(f"w_ex = {w_ex}")
r_ex = tensor_vector_with_multilinear_basis(r_vec)
print(f"r_ex = {r_ex}")
scalar = inner_product(r_ex, w_ex, Fp(0))
scalar_2 = reduce(mul, [inner_product([Fp(1)-r_vec[j], r_vec[j]], [1, w_rev[k_last:][j]], Fp(0)) for j in range(len(r_vec))], Fp(1))
print(f"scalar = {scalar}, scalar_2 = {scalar_2}")
assert scalar == scalar_2, f"scalar != scalar_2, {scalar} != {scalar_2}"

new_eq = tensor_vector_with_monomial_basis(w_rev[:k_last])
new_eq = [new_eq[j] * scalar for j in range(2**k_last)]
eq_2 = [eq_0[j] + beta * new_eq[j] for j in range(2**k_last)]
eq_2, new_eq, scalar


omega = 9
vec = [9, 81]
w_rev = [9, 81]
w_ex = [1, 81]
r_ex = [118, 76]
scalar = 98, scalar_2 = 98


([6, 91], [98, 110], 98)

In [162]:
# iteration 1

q=0
k1_prime = 1
k1 = 1
r_vec = r_vec_all[:k1-k_last]
beta = Fp(12)

w_rev = tensor_vector(q, k1, 2)
print(f"w_rev = {w_rev}")
w_ex = tensor_vector_with_monomial_basis(w_rev[k_last:])
print(f"w_ex = {w_ex}")
r_ex = tensor_vector_with_multilinear_basis(r_vec)
print(f"r_ex = {r_ex}")
scalar = inner_product(r_ex, w_ex, Fp(0))
scalar_2 = reduce(mul, [inner_product([Fp(1)-r_vec[j], r_vec[j]], [1, w_rev[k_last:][j]], Fp(0)) for j in range(len(r_vec))], Fp(1))
print(f"scalar = {scalar}, scalar_2 = {scalar_2}")
assert scalar == scalar_2, f"scalar != scalar_2, {scalar} != {scalar_2}"

new_eq = tensor_vector_with_monomial_basis(w_rev[:k_last])
new_eq = [new_eq[j] * scalar for j in range(2**k_last)]
eq_3 = [eq_2[j] + beta * new_eq[j] for j in range(2**k_last)]
eq_3, new_eq, scalar

omega = 1
vec = [1]
w_rev = [1]
w_ex = [1]
r_ex = [1]
scalar = 1, scalar_2 = 1


([18, 103], [1, 1], 1)